# Black Litterman Model in Python

The Black Litterman procedure can be viewed as a bayesian shrinkage method, that shrinks the expected returns constructed from an investor's views on asset returns towards asset returns implied by the market equilibrium. The procedure computes a set of expected returns that uses the market equilibrium implied  as a prior. This is then combined with returns implied by subjective investor views to produce a set of posterior expected returns $\mu^{BL}$ and covariances $\Sigma^{BL}$.

The obvious attraction of being able to incorporate subjective investor views, the Black-Litterman procedure has a second feature that makes it extremely attractive to portfolio optimization. It is well known that the Markowitz optimization procedure is highly sensitive to estimation errors in Expected Returns and Covariances, and this error maximizing nature of the Markowitz procedure causes unstable portfolios with extreme weights that diverge rapidly from the market equilibrium portfolio even with minor changes to the inputs. However, the posterior parameters $\mu^{BL}, \Sigma^{BL}$ computed by the Black Litterman procedure are derived in part from the market portfolio, and therefore are much more pragmatic inputs for purposes of portfolio optimization. 

\begin{array}{ll}
Q & \mbox{An $K \times 1$ ''Qualitative Views'' or simply, Views matrix} \\
P & \mbox{A $K \times N$ ''Projection'' or ''Pick'' matrix, linking each view to the assets} \\
\Omega & \mbox{A Covariance matrix representing the uncertainty of views}
\end{array}


If the $k$-th views is an absolute view, it is represented by setting $Q_k$ to the expected return of asset $k$ and setting $P_{ki}$ to 1 and all other elements of row $k$ in $P$ to zero

If $k$-th view is a relative view, between asset $i$ and $j$ it is represented by setting $Q_k$ to the expected difference of returns between assets $i$ and $j$, and setting $P_{ki}$ to **-1** for the underperforming asset, $P_{kj}$ to **+1** and all other elements of row $k$ in $P$ to zero. $\Omega$ is either set to the specified uncertainty or is inferred from the user or from the data. HE suggess setting $\Omega$ to be the diagonal matrix obtained from the diagonal elements of $P \tau \Sigma P^T$, which is what we shall do for our initial tests.

#### The Master Formula

The first step of the procedure is a _reverse-optimization_ step that infers the implied returns vector $\pi$ that are implied by the equilibrium weights $w$ using the formula:

$$\pi = \delta\Sigma w$$

Next, the posterior returns and covariances are obtained from the _Black-Litterman Master Formula_ which is the following set of equations:

\begin{equation}
\label{eq:blMuOrig}
\mu^{BL} = [(\tau\Sigma)^{-1} + P \Omega^{-1} P]^{-1}[(\tau\Sigma)^{-1} \pi + P \Omega^{-1} Q]
\end{equation}

\begin{equation}
\label{eq:blSigmaOrig}
\Sigma^{BL} = \Sigma + [(\tau\Sigma)^{-1} + P \Omega^{-1} P]^{-1}
\end{equation}

#### Inverting $\Omega$

While the master formulas identified in Equation \ref{eq:blMuOrig} and Equation \ref{eq:blSigmaOrig} are frequently easy to implement, they do involve the term $\Omega^{-1}$. Unfortuantely, $\Omega$ is sometimes non-invertible, which poses difficulties to implement the equations as-is. Fortunately the equations are easily transformed to a form that does not require this troublesome inversion. Therefore, frequently, implementations use the following equivalent versions of these equations which are sometimes computationally more stable, since they do not involve inverting $\Omega$. Derivations of these alternate forms are provided in the appendices of \cite{walters2011black}:

\begin{equation}
\label{eq:blMu}
\mu^{BL} = \pi + \tau \Sigma P^T[(P \tau \Sigma P^T) + \Omega]^{-1}[Q - P \pi]
\end{equation}

\begin{equation}
\label{eq:blSigma}
\Sigma^{BL} = \Sigma + \tau \Sigma - \tau\Sigma P^T(P \tau \Sigma P^T + \Omega)^{-1} P \tau \Sigma
\end{equation}


In [1]:
import numpy as np 
import pandas as pd

In [2]:
def as_colvec(x):
    """
    Helper function. returns the data as a column vector
    """
    if (x.ndim == 2):
        return x
    else:
        return np.expand_dims(x,axis=1)

In [3]:
np.arange(4)

array([0, 1, 2, 3])

In [4]:
as_colvec(np.arange(4))

array([[0],
       [1],
       [2],
       [3]])

Recall that the first step is to reverse engineer the implied retrns vector $\pi$ from a set of portfolio weights $W$

$\pi$ = $\delta \Sigma W$

In [5]:
def implied_returns(delta, sigma,w):
    """
    Obtain the implied returns by reverse engineering the weights
    Inputs:
    delta: risk aversion coef (scalar)
    sigma: Variance-Covar matrix (N x N) as DataFrame
    w: portflio weights of returns as Series
    Returns an N x 1 vector of returns as Series
    """
    ir = delta * sigma.dot(w).squeeze() # to get a series from a 1-col dataframe
    ir.name = 'Implied Returns'
    return ir

If the investor cannot have a specific way to explicitly quantify the uncertainly associated with the view in the $\Omega$ matrix, one can make the assumption that $\Omega$ is propotional to the variance of the prior.
Specifically, they suggest that:

$\Omega$ = $diag(P(\tau \Sigma)P^T)$

In [6]:
#Assume that Omega is propotional to the variance of the prior
def propotional_prior(sigma,tau,p):
    """
    Returns the He-Litterman simplified Omega
    Inputs:
    sigma: N x N covar matrix as DataFrame
    tau: a scalar
    p: K x N DataFrame linking to Q and assets
    returns a P x P dataframe, a Marix representing Prior Uncertainties
    """
    helit_omega = p.dot(tau * sigma).dot(p.T)
    # Makes a diag matrix from the diag elements of Omega
    return pd.DataFrame(np.diag(helit_omega.values),index=p.index,columns=p.index)

In [7]:
# We use this function to compute the posterior expected returns as follows

from numpy.linalg import inv

def bl(w_prior, sigma_prior, p,q,omega=None,delta=2.5,tau=0.02):
    """
    Computes the posterior expected returns based on the original black litterman reference model
    W.prior must be an N x 1 vector of weights, a Series
    Sigma.prior is an N x N covariance matrix, a DataFrame
    P must be a K x N matrix linking Q and the Assets, a DataFrame
    Q must be an K x 1 vector of views, a Series
    Omega must be a K x K matrix a DataFrame, or None
    if Omega is None, we assume it is proportional to variance of the prior 
    delta and tau are scalars
    """
    if omega is None:
        omega = propotional_prior(sigma_prior,tau,p)
    
    #How many assets do we have?
    N = w_prior.shape[0]
    #And how many views?
    K = q.shape[0]

    #First reverse engineering the weights to get the pi
    pi = implied_returns(delta,sigma_prior,w_prior)
    #Adjust Sigma by the uncertainty scaling factor
    sigma_prior_scaled = tau * sigma_prior
    # posterior estimate of te mean, use the 'Master Formula'
    # We use the versions that do not require Omega to be inverted
    #This is easier to read if we use @ for matrixmult instead of .dot()
    # mu_bl = pi + sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ (q - p @ pi)
    mu_bl = pi + sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega).dot(q - p.dot(pi).values))
    # posterior estimate of uncertainty of mu.bl
    # sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ p @ sigma_prior_scaled
    sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega)).dot(p).dot(sigma_prior_scaled)
    return (mu_bl, sigma_bl)
    

A simpe example: Absolute views
we start with a siple 2-asset example. Let's start with an example from *Statistical Models and Methods for Financial Markets*
Consider the portfolio consisting of 2 stocks: Intel and Pfizer
From the book Table 3.1 on page 72, we obtain the covar matrix(multiplied by $10^4$)

Assume that Interl has a market cap of approximately USD 80B and that of Pfizer is approx USD 100B (this is not required accurate, but works just fine as an example) Thus, if you held a market-cap weighted portfolio you would hold INTC and PFE with the following weights: *$W_{INTC}$* = 80/180 = 44%, *$W_{PFE}$* = 100/180 = 56%. These appear to be reasonable weights wiyhout an extreme allocation to either stock, even though Pfizer is slightly overweighted.

We can compute the equilibruim implied returns $\Pi$ as follows:

In [8]:
tickers = ['INTC','PFE']
s = pd.DataFrame([[46,1.06],[1.06,5.33]],index=tickers,columns=tickers)* 10E-4
pi = implied_returns(delta=2.5,sigma=s,w=pd.Series([.44,.56],index=tickers))
pi

INTC    0.052084
PFE     0.008628
Name: Implied Returns, dtype: float64

Thus the equilibrium implied returns for INTC are a bit more than 5% and abit less than 1% for PFE.

Assume that investors think that Intel will return 2% anf that Pfizer will return 4%. We can now examine the optimal weights according to the Markowitz procedure. Waht would happen if we use these expected returns to compute the optial max sharp ratio portfolio?

The Max Sharp Ratio Portfolio weights are easily computed in explicit form if there are no constraints on the weights. The weights are given by the expression:

$$ W_{MSR} = \frac{\Sigma^{-1}\mu_e}{\bf{1}^T \Sigma^{-1}\mu_e} $$

where *$\mu_e$* is the vector of expected returns and $\Sigma$ is the var-covar matrix.

This is implemented as follows: